In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Required imports definition
import cv2
import os
import numpy as np
from PIL import Image
import skimage
from scipy.ndimage import gaussian_filter
from skimage import measure, morphology
from tqdm.notebook import tqdm
import tensorflow as tf
from keras import layers, models, metrics, losses, callbacks, optimizers


In [ ]:
# Include the name of your group
group_name = 'FMZ12'

In [ ]:
zip_file_path="/content/drive/MyDrive/EIMproject/RETINA/RETINA.zip"
destination_folder="/content/RETINA"
import zipfile
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

In [ ]:
#################### INSTRUCTIONS FOR FOLDER MANAGEMENT ########################
#
#
# N.B ::: Leave this as it is, we will input the required paths
#

submission_folder = 'Submission_FMZ12'
print(f'submission folder: {submission_folder}')

model_path = os.path.join(submission_folder, "unet3plus_gs_ds.h5")

test_img_folder = '...'

results_folder = os.path.join(submission_folder, f'results_{group_name}')

if not os.path.exists(results_folder):
    os.makedirs(results_folder)

submission folder: /content/drive/MyDrive/EIMproject/Submission_FMZ12


In [ ]:
#Le seguenti funzioni sono necessarie per far funzionare l'allenamento in quanto è stata usata una custom loss nella fase di training,
#devono essere inizializzate per ottenere le maschere automatiche, altimenti non funziona
#NON SERVONO A CALCOLARE LE METRICHE
def digitize(y, treshold=0.5):
  return tf.cast(tf.greater_equal(y, treshold), tf.float32)

def IoU(y_true, y_pred):
  y_true = digitize(y_true)
  y_pred = digitize(y_pred)
  intersection = tf.multiply(y_true, y_pred)
  suminter = tf.reduce_sum(intersection)
  return suminter / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - suminter)

def dice_coefficient(y_true, y_pred):
  y_true = digitize(y_true)
  y_pred = digitize(y_pred)
  intersection = tf.multiply(y_true, y_pred)
  suminter = tf.reduce_sum(intersection)
  return 2 * suminter / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred))

def dice_loss(y_true, y_pred, smooth=1):
  intersection = tf.multiply(y_true, y_pred)
  return 1 - (2 * tf.reduce_sum(intersection) + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def iou_loss(y_true, y_pred, smooth=1):
  intersection =  tf.reduce_sum(tf.multiply(y_true, y_pred))
  return 1 - (intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection + smooth)

def mae(y_true, y_pred):
  return tf.reduce_mean(tf.abs(y_true - y_pred))

def mse(y_true, y_pred):
  return tf.reduce_mean(tf.square(y_true - y_pred))

def custom_loss(losses, weights):
  def loss(y_true, y_pred):
    value = tf.constant([0.0])
    for l, w in zip(losses, weights):
      value += l(y_true, y_pred) * w
    return value
  return loss

#Caricamento del modello
model = models.load_model(model_path,
                          custom_objects={
                                 'loss': custom_loss([dice_loss, iou_loss], [1, 1]),
                                 'IoU': IoU,
                                 'dice_coefficient': dice_coefficient})

**IMPORTANT: rules for the inference step**

Below there is an example of the loop to perform the inference. Edit this loop to implement your pre-processing, inference and post-processing strategies. Don't save the images resulting from intermediate steps. Save only the final prediction (0-background, 255-objects). Don't compute the metrics on the results, we will compute the metrics on your results in a separate notebook.

Please look at how preprocessing is performed. Only one temporary file is saved at each cycle, this allows us to perform grading of all submissions without filling Colab memory. When including your strategies, please mind this aspect.


In [ ]:
#Inizializzazione dell'oggetto CLAHE
clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))

#Inizializzazione del kernel per lo sharpening
kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])


def preprocessing(original):

  #Salavataggio dell'informazione sulla dimensione di partenza
    original_width, original_height=original.size
  #Applicazione del resize a 512x512
    im_resized=np.array(original.resize((512,512)))
  #Estrazione del canale verde dall'immagine
    green=im_resized[:,:,1]

  #Calcolo della moda dal canale verde
    unique_values, counts = np.unique(green, return_counts=True)
    #Estrazione dell'indice del massimo dei counts
    max_count_index = np.argmax(counts)
    #La moda è il valore all'indice del massimo dei counts
    mode_value = unique_values[max_count_index]
  #Definizione della maschera come area dove l'immagine ha valori maggiori della moda +2
    mask_cerchio = green > mode_value +2
  #Applicazione dell'apertura per rimuovere eventuali residui bianchi
    mask_cerchio=morphology.opening(mask_cerchio,footprint=np.ones((16,16)))
    mask_cerchio=mask_cerchio.astype(bool)
  #I valori dell'immagine al di fuori della maschera sono posti a zero
    green[~mask_cerchio]=0
  #Applicazione del CLAHE
    cl1 = clahe.apply(green)

  #Applicazione dello sharpening
    sharpened_image = cv2.filter2D(cl1, -1, kernel)
  #Standardizzazione
    std_im = (sharpened_image - np.mean(sharpened_image)) / np.std(sharpened_image)
  #Normalizzazione
    norm_im=(std_im-np.min(std_im))/(np.max(std_im)-np.min(std_im))

    return norm_im,original_width,original_height

In [ ]:
# Loop over the test images
for img_name in tqdm(os.listdir(test_img_folder)):
    if img_name.endswith('.png'):

      img_path = os.path.join(test_img_folder, img_name)

      ###############################
      ####### Pre-Processing ########
      ###############################
    #Caricamento dell'immagine
      image=Image.open(img_path)

    #Applicazione del pre-processing
      norm_im,width,height=preprocessing(original=image)


      ###############################
      #######    Inference   ########
      ###############################
    #Predizione della maschera automatica con la rete
      #N.B. Per via della deep supervsion, abbiamo un output per ogni stage del decoder, prendiamo solo quella finale
      img_pred = model.predict(np.expand_dims(norm_im, axis=0))[0][0]

    #Resize dell'immagine alle dimesioni precedenti
      img_inf_res = cv2.resize(img_pred, dsize=(width,height), interpolation=cv2.INTER_CUBIC)
    #Per via dell'interpolazione, settiamo una soglia per determinare i valori intermedi
      img_inf_res[img_inf_res > 0.4] =1
      img_inf_res[img_inf_res <= 0.4] =0
    #Maschera riscalata tra 0 e 255 e convertita in uint8
      img_inf_res=(img_inf_res*255).astype(np.uint8)
      ###############################
      ####### Post-Processing #######
      ###############################

    # Il nostro post-process si applica solo al calcolo dell'albero vascolare,
    # Applicare alle immagini salavate la funzione salvata nel file branch_analysis_FMZ12.py

      ###############################
      #######   Save results  #######
      ###############################

    #Conversione ad immagine
      pred_label = Image.fromarray(img_inf_res)
    # Salvataggio della maschera predetta
      pred_label.save(os.path.join(results_folder, img_name))


  0%|          | 0/600 [00:00<?, ?it/s]

1/1 [==============================] - 0s 25ms/step


KeyboardInterrupt: 